# Data Module

> `DataModule` for training parametric models, generating and benchmarking CF explanations.

In [ ]:
#| default_exp data_module

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.utils import load_json, validate_configs, cat_normalize, get_config
from relax.base import *
from relax.data_utils import *
import jax
from jax import numpy as jnp, random as jrand, lax, Array
import pandas as pd
import numpy as np
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
# from sklearn.base import TransformerMixin
# from sklearn.utils.validation import check_is_fitted, NotFittedError
from urllib.request import urlretrieve
# from relax.data.loader import Dataset, ArrayDataset, DataLoader, DataloaderBackends
from pydantic.fields import ModelField, Field
from typing import List, Dict, Union, Optional, Tuple, Callable, Any, Iterable

2023-07-21 10:15:57.546893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using JAX backend.


In [ ]:
#| hide
from fastcore.test import *
from copy import deepcopy

## Data Module Interfaces

High-level interfaces for `DataModule`. Docs to be added. 

In [ ]:
#| export
class BaseDataModule(BaseModule):
    """DataModule Interface"""

    def prepare(self, *args, **kwargs):
        """Prepare data for training"""
        raise NotImplementedError
        
    def apply_constraints(self, x: Array, cf: Array, hard: bool = False, **kwargs) -> Array:
        raise NotImplementedError
    
    def apply_regularization(self, x: Array, cf: Array, hard: bool = False, **kwargs) -> float:
        raise NotImplementedError

## Tabula Data Module

`DataModule` for processing tabular data.


In [ ]:
#| export
class DataModuleConfig(BaseConfig):
    """Configurator of `TabularDataModule`."""

    data_dir: str = Field(description="The directory of dataset.")
    data_name: str = Field(description="The name of `DataModule`.")
    continous_cols: List[str] = Field([], description="Continuous features/columns in the data.")
    discret_cols: List[str] = Field([], description="Categorical features/columns in the data.")
    imutable_cols: List[str] = Field([], description="Immutable features/columns in the data.")
    continuous_transformation: str = Field('minmax', description="Transformation for continuous features.")
    discret_transformation: str = Field('ohe', description="Transformation for categorical features.")
    sample_frac: Optional[float] = Field(
        None, description="Sample fraction of the data. Default to use the entire data.", ge=0., le=1.0
    )
    train_indices: List[int] = Field([], description="Indices of training data.")
    test_indices: List[int] = Field([], description="Indices of testing data.")
    
    # normalizer: Optional[str] = Field(
    #     default_factory=lambda: MinMaxScaler(),
    #     description="Sklearn scalar for continuous features. Can be unfitted, fitted, or None. "
    #     "If not fitted, the `TabularDataModule` will fit using the training data. If fitted, no fitting will be applied. "
    #     "If `None`, no transformation will be applied. Default to `MinMaxScaler()`."
    # )
    # encoder: Optional[str] = Field(
    #     default_factory=lambda: OneHotEncoder(sparse=False),
    #     description="Fitted encoder for categorical features. Can be unfitted, fitted, or None. "
    #     "If not fitted, the `TabularDataModule` will fit using the training data. If fitted, no fitting will be applied. "
    #     "If `None`, no transformation will be applied. Default to `OneHotEncoder(sparse=False)`."
    # )

    def shuffle(self, data: Array, test_size: float, seed: int = None):
        """Shuffle data with a seed."""
        if seed is None:
            seed = get_config().global_seed
        key = jrand.PRNGKey(seed)
        total_length = data.shape[0]
        train_length = int((1 - test_size) * total_length)
        self.train_indices = jrand.permutation(key, total_length)[:train_length].tolist()
        self.test_indices = jrand.permutation(key, total_length)[train_length:].tolist()

In [ ]:
config = DataModuleConfig(data_name="TabularDataModule", data_dir="data", continous_cols=[], discret_cols=[], imutable_cols=[])
config.shuffle(data=np.arange(100), test_size=0.2)
assert len(config.train_indices) == 100 * 0.8
assert len(config.test_indices) == 100 * 0.2
assert isinstance(config.train_indices, list)
assert isinstance(config.test_indices, list)

configs_dict = {
    "data_dir": "assets/data/s_adult.csv",
    "data_name": "adult",
    "continous_cols": ["age", "hours_per_week"],
    "discret_cols": ["workclass", "education", "marital_status","occupation"],
    "imutable_cols": ["age", "workclass", "marital_status"],
    "sample_frac": 0.1,
}
configs = DataModuleConfig(**configs_dict)
assert len(configs.train_indices) == 0
assert len(configs.test_indices) == 0
assert config.continuous_transformation == 'minmax'
assert config.discret_transformation == 'ohe'

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [ ]:
#| export
class DataModule(BaseDataModule):
    def __init__(
        self, 
        config, 
        data: pd.DataFrame = None,
        features: List[Feature] = None,
        label: Feature = None,
    ):
        config = validate_configs(config, DataModuleConfig)
        if data is None:
            data = pd.read_csv(config.data_dir)
        self._data = data
        features = self.convert_to_features(config, data, features)
        label = self.convert_to_label(config, data, label)
        self.prepare(features, label)
        config.shuffle(self.xs, test_size=0.25)
        super().__init__(config, name=config.data_name)

    def convert_to_features(
        self, 
        config: DataModuleConfig, 
        data: pd.DataFrame, 
        features: list[Feature] = None
    ):
        to_feature = lambda col, data, is_continuous: Feature(
                name=col, data=data[col].to_numpy().reshape(-1, 1),
                transformation=config.continuous_transformation if is_continuous else config.discret_transformation,
                is_immutable=col in config.imutable_cols
            )

        if features is not None:
            return features
        
        cont_features = [to_feature(col, data, True) for col in config.continous_cols]
        cat_features = [to_feature(col, data, False) for col in config.discret_cols]
        return cont_features + cat_features        
        
    def convert_to_label(self, config: DataModuleConfig, data: pd.DataFrame, label: Feature = None):
        if label is not None:
            return label
        
        label_col = data.columns[-1]
        return Feature(
            name=label_col, data=data[label_col].to_numpy().reshape(-1, 1),
            transformation='identity',
            is_immutable=label_col in config.imutable_cols
        )
        
    def prepare(self, features, label):
        if features is not None and label is not None:
            self._features = FeaturesList(features)
            self._label = label
        elif features is None:
            raise ValueError("Features cannot be None.")
        elif label is None:
            raise ValueError("Label cannot be None.")
    
    @property
    def data(self) -> pd.DataFrame:
        return self._data
    
    @property
    def xs(self) -> Array:
        return self._features.transformed_data
    
    @property
    def ys(self) -> Array:
        return self._label.transformed_data

    @property
    def dataset(self) -> Tuple[Array, Array]:
        return (self.xs, self.ys)
    
    def _get_data(self, indices):
        if isinstance(indices, list):
            indices = jnp.array(indices)
        return (self.xs[indices], self.ys[indices])
        
    def __getitem__(self, name: str):
        if name == 'train':
            return self._get_data(self.config.train_indices)
        elif name in ['valid', 'test']:
            return self._get_data(self.config.test_indices)
        else:
            raise ValueError(f"Unknown data name: {name}. Should be one of ['train', 'valid', 'test']")
        
    def apply_constraints(self, x: Array, cf: Array, hard: bool = False) -> Array:
        return self._features.apply_constraints(x, cf, hard)

In [ ]:
dm = DataModule(configs)
xs = dm['train'][0]
cfs = jrand.uniform(jrand.PRNGKey(0), shape=xs.shape, minval=0.01, maxval=0.99)
cfs = dm.apply_constraints(xs, cfs, hard=False)
assert cfs.shape == xs.shape

cfs = dm.apply_constraints(xs, cfs, hard=True)
assert cfs.shape == xs.shape


## Load Data


In [ ]:
#| exporti
DEFAULT_DATA_CONFIGS = {
    'adult': {
        'data' :'assets/adult/data.csv',
        'conf' :'assets/adult/configs.json',
        'model' :'assets/adult/model'
    },
    'heloc': {
        'data': 'assets/heloc/data.csv',
        'conf': 'assets/heloc/configs.json',
        'model' :'assets/heloc/model'
    },
    'oulad': {
        'data': 'assets/oulad/data.csv',
        'conf': 'assets/oulad/configs.json',
        'model' :'assets/oulad/model'
    },
    'credit': {
        'data': 'assets/credit/data.csv',
        'conf': 'assets/credit/configs.json',
        'model' :'assets/credit/model'
    },
    'cancer': {
        'data': 'assets/cancer/data.csv',
        'conf': 'assets/cancer/configs.json',
        'model' :'assets/cancer/model'
    },
    'student_performance': {
        'data': 'assets/student_performance/data.csv',
        'conf': 'assets/student_performance/configs.json',
        'model' :'assets/student_performance/model'
    },
    'titanic': {
        'data': 'assets/titanic/data.csv',
        'conf': 'assets/titanic/configs.json',
        'model' :'assets/titanic/model'
    },
    'german': {
        'data': 'assets/german/data.csv',
        'conf': 'assets/german/configs.json',
        'model' :'assets/german/model'
    },
    'spam': {
        'data': 'assets/spam/data.csv',
        'conf': 'assets/spam/configs.json',
        'model' :'assets/spam/model'
    },
    'ozone': {
        'data': 'assets/ozone/data.csv',
        'conf': 'assets/ozone/configs.json',
        'model' :'assets/ozone/model'
    },
    'qsar': {
        'data': 'assets/qsar/data.csv',
        'conf': 'assets/qsar/configs.json',
        'model' :'assets/qsar/model'
    },
    'bioresponse': {
        'data': 'assets/bioresponse/data.csv',
        'conf': 'assets/bioresponse/configs.json',
        'model' :'assets/bioresponse/model'
    },
    'churn': {
        'data': 'assets/churn/data.csv',
        'conf': 'assets/churn/configs.json',
        'model' :'assets/churn/model'
    },
    'road': {
        'data': 'assets/road/data.csv',
        'conf': 'assets/road/configs.json',
        'model' :'assets/road/model'
    }
}

In [ ]:
pd.read_csv(DEFAULT_DATA_CONFIGS['heloc']['data'])['MaxDelq2PublicRecLast12M'].to_numpy(na_value=np.nan)

array([nan, '30 Days Delinquent', nan, ..., 'Derogatory Comment',
       'Never Delinquent', nan], dtype=object)

In [ ]:
for data_name in DEFAULT_DATA_CONFIGS.keys():
    print(f"Loading {data_name}...")
    conf_path = DEFAULT_DATA_CONFIGS[data_name]['conf']
    config = load_json(conf_path)['data_configs']
    dm_config = DataModuleConfig(**config)
    dm = DataModule(dm_config)

Loading adult...
Loading heloc...
Loading oulad...
Loading credit...
Loading cancer...
Loading student_performance...
Loading titanic...
Loading german...
Loading spam...
Loading ozone...
Loading qsar...
Loading bioresponse...
Loading churn...
Loading road...


In [ ]:
config = load_json('assets/adult/configs.json')['data_configs']
dm_config = DataModuleConfig(**config)
dm = DataModule(dm_config)

In [ ]:
#| exporti
def _validate_dataname(data_name: str):
    if data_name not in DEFAULT_DATA_CONFIGS.keys():
        raise ValueError(f'`data_name` must be one of {DEFAULT_DATA_CONFIGS.keys()}, '
            f'but got data_name={data_name}.')

In [ ]:
#| export
def load_data(
    data_name: str, # The name of data
    return_config: bool = False, # Return `data_config `or not
    data_configs: dict = None # Data configs to override default configuration
) -> TabularDataModule | Tuple[TabularDataModule, TabularDataModuleConfigs]: 
    """High-level util function for loading `data` and `data_config`."""
    
    _validate_dataname(data_name)

    # get data/config urls
    _data_path = DEFAULT_DATA_CONFIGS[data_name]['data']
    _conf_path = DEFAULT_DATA_CONFIGS[data_name]['conf']
    
    data_url = f"https://github.com/BirkhoffG/ReLax/raw/master/{_data_path}"
    conf_url = f"https://github.com/BirkhoffG/ReLax/raw/master/{_conf_path}"

    # create new dir
    data_dir = Path(os.getcwd()) / "cf_data" / data_name
    if not data_dir.exists():
        os.makedirs(data_dir)
    data_path = data_dir / 'data.csv'
    conf_path = data_dir / 'configs.json'

    # download data/configs
    if not data_path.is_file():
        urlretrieve(data_url, data_path)    
    if not conf_path.is_file():
        urlretrieve(conf_url, conf_path)

    # read config
    config = load_json(conf_path)['data_configs']
    config['data_dir'] = str(data_path)

    if not (data_configs is None):
        config.update(data_configs)

    config = TabularDataModuleConfigs(**config)
    data_module = TabularDataModule(config)

    if return_config:
        return data_module, config
    else:
        return data_module


`load_data` easily loads example datasets by passing the `data_name`. 
For example, you can load the [adult](https://archive.ics.uci.edu/ml/datasets/adult) as:

In [ ]:
dm = load_data(data_name = 'adult')

Underlying, `load_data` loads the default `data_configs`. To access this `data_configs`,

In [ ]:
dm, data_configs = load_data(data_name = 'adult', return_config=True)

If you want to override some of the data configs, 
you can pass it as an auxillary argumenet in `data_configs`. 
For example, if you want to use only 10% of the data, you can

In [ ]:
dm = load_data(
    data_name = 'adult', data_configs={'sample_frac': 0.1}
)

#### Supported Datasets

`load_data` currently supports following datasets:

In [ ]:
#|echo: false
def display_data_attrbutes(names: list):
    attrs = {
        '# Cont Features': { data_name: 0 for data_name in names}, 
        '# Cat Features': { data_name: 0 for data_name in names},
        '# of Data Points': { data_name: 0 for data_name in names}, 
    }
    for data_name in names:
        dm, config = load_data(data_name, return_config=True)
        attrs['# Cont Features'][data_name] = len(config.continous_cols)
        attrs['# Cat Features'][data_name] = len(config.discret_cols)
        attrs['# of Data Points'][data_name] = len(dm.data)

        # run tests
        check_datamodule(dm, config)
    return pd.DataFrame.from_dict(attrs)

display_data_attrbutes(DEFAULT_DATA_CONFIGS.keys())

# Cont Features  # Cat Features  # of Data Points
adult                              2               6             32561
heloc                             21               2             10459
oulad                             23               8             32593
credit                            20               3             30000
cancer                            30               0               569
student_performance                2              14               649
titanic                            2              24               891
german                             7              13              1000
spam                              57               0              4601
ozone                             72               0              2534
qsar                              38               3              1055
bioresponse                     1776               0              3751
churn                              3              16              7043
road                              29               3            111762

In [ ]:
DEFAULT_DATA_CONFIGS.keys()

dict_keys(['adult', 'heloc', 'oulad', 'credit', 'cancer', 'student_performance', 'titanic', 'german', 'spam', 'ozone', 'qsar', 'bioresponse', 'churn', 'road'])

In [ ]:
#| hide
for data_name in DEFAULT_DATA_CONFIGS.keys():
    dm, config = load_data(
        data_name, return_config=True, data_configs=dict(sample_frac=0.1)
    )
    assert config.sample_frac == 0.1
    check_datamodule(dm, config)